In [43]:
%reset -f

In [44]:
import pandas as pd

import requests

from bs4 import BeautifulSoup

import wget

import os

import codecs

import sys

In [45]:
def getDirectoryName(dirName):
    dirName = dirName.replace(',', '')
    dirName = dirName.replace(' ', '_')
    dirName = 'SpringerBooks_'+dirName
    return dirName

In [46]:
df = pd.read_excel('Free+English+textbooks.xlsx')

colsToDrop = ['Product Type', 'Copyright Year', 'Copyright Holder', 'Edition', 'Print ISBN', 'Electronic ISBN', 'Language Collection', 'German Package Name',
              'Series Print ISSN', 'Series Electronic ISSN', 'Series Title', 'Volume Number', 'DOI URL', 'Subject Classification', 'Publisher', 'Imprint', 'eBook Package', 'Language']

df = df.drop(colsToDrop, axis=1)

df['DirectoryName'] = [getDirectoryName(i) for i in df['English Package Name']]

df.head()

,Book Title,Author,English Package Name,OpenURL,DirectoryName
0,Fundamentals of Power Electronics,"Robert W. Erickson, Dragan Maksimovic",Engineering,http://link.springer.com/openurl?genre=book&is...,SpringerBooks_Engineering
1,Handbook of the Life Course,"Jeylan T. Mortimer, Michael J. Shanahan","Humanities, Social Sciences and Law",http://link.springer.com/openurl?genre=book&is...,SpringerBooks_Humanities_Social_Sciences_and_Law
2,All of Statistics,Larry Wasserman,Mathematics and Statistics,http://link.springer.com/openurl?genre=book&is...,SpringerBooks_Mathematics_and_Statistics
3,Social Anxiety and Social Phobia in Youth,Christopher Kearney,Behavioral Science,http://link.springer.com/openurl?genre=book&is...,SpringerBooks_Behavioral_Science
4,Discrete Mathematics,"László Lovász, József Pelikán, Katalin Veszter...",Mathematics and Statistics,http://link.springer.com/openurl?genre=book&is...,SpringerBooks_Mathematics_and_Statistics


In [47]:
# create the directories for the book types (if they don't already exist)

for dr in set(df['DirectoryName']):
    if os.path.exists(dr):
        'Do Nothing'
    else:
        os.mkdir(dr)

In [56]:
def getSpringerBookPDF(tupleBookDetails):
    booksFolder = tupleBookDetails[4]
    bookURL = tupleBookDetails[3]
    theFileName = tupleBookDetails[0]+' ['+tupleBookDetails[1]+']'
    print('Saving "' + theFileName.upper() + '" to ' + booksFolder)
    thisPage = requests.get(bookURL)
    thisSoup = BeautifulSoup(thisPage.content, "html.parser")
    thisItem = thisSoup.find('a', {'title': 'Download this book in PDF format'})
    theLinkedPDF = 'http://link.springer.com/' + thisItem['href']
    wget.download(theLinkedPDF, "{}/{}.pdf".format(booksFolder, theFileName))
    print('\n\tDONE\n')

In [57]:
for tp in list(zip(df['Book Title'], df['Author'], df['English Package Name'], df['OpenURL'], df['DirectoryName']))[:5]:
    getSpringerBookPDF(tp)

Saving "FUNDAMENTALS OF POWER ELECTRONICS [ROBERT W. ERICKSON, DRAGAN MAKSIMOVIC]" to SpringerBooks_Engineering


FileNotFoundError: [Errno 2] No such file or directory: '.\\SpringerBooks_Engineering/Fundamentals of Power Electronics [Robert W. Erickson, Dragan Maksimovic].pdf6oqdro2s.tmp'